# **📄 Document type classification baseline code**
> 문서 타입 분류 대회에 오신 여러분 환영합니다! 🎉     
> 아래 baseline에서는 ResNet 모델을 로드하여, 모델을 학습 및 예측 파일 생성하는 프로세스에 대해 알아보겠습니다.

## Contents
- Prepare Environments
- Import Library & Define Functions
- Hyper-parameters
- Load Data
- Train Model
- Inference & Save File


## 1. Prepare Environments

* 데이터 로드를 위한 구글 드라이브를 마운트합니다.
* 필요한 라이브러리를 설치합니다.

## 2. Import Library & Define Functions
* 학습 및 추론에 필요한 라이브러리를 로드합니다.
* 학습 및 추론에 필요한 함수와 클래스를 정의합니다.

In [18]:
import gc
gc.collect()

1525

In [19]:
import os
import time
import random

import timm
import torch
import albumentations as A
import pandas as pd
import numpy as np
import torch.nn as nn
from albumentations.pytorch import ToTensorV2
from torch.optim import Adam
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader, Subset
from PIL import Image
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score

In [20]:
import json
import wandb
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay # sklearn 내 confusion matrix 계산 함수
import matplotlib.pyplot as plt # 시각화를 위한 라이브러리
import torchvision.transforms as T # 이미지 변환을 위한 모듈

In [21]:
# 시드를 고정합니다.
SEED = 42
os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.benchmark = True

In [22]:
import os
import pandas as pd
import numpy as np
from PIL import Image
from torch.utils.data import Dataset

class ImageDataset(Dataset):
    def __init__(self, csv, path, transform=None):
        self.df = pd.read_csv(csv).values
        self.path = path
        self.transform = transform
        self.image_paths = self._get_all_image_paths()

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        name, target = self.df[idx]
        img_path = self.image_paths.get(name)
        if img_path is None:
            raise FileNotFoundError(f"Image {name} not found in {self.path}")
        img = np.array(Image.open(img_path))
        if self.transform:
            img = self.transform(image=img)['image']
        return img, target

    def _get_all_image_paths(self):
        image_paths = {}
        for root, _, files in os.walk(self.path):
            for file in files:
                if file.endswith(('.jpg', '.jpeg', '.png')):
                    image_paths[file] = os.path.join(root, file)
        return image_paths

In [23]:
# one epoch 학습을 위한 함수입니다.
def train_one_epoch(loader, model, optimizer, loss_fn, device):
    model.train()
    train_loss = 0
    preds_list = []
    targets_list = []

    pbar = tqdm(loader)
    for image, targets in pbar:
        image = image.to(device)
        targets = targets.to(device)

        model.zero_grad(set_to_none=True)

        preds = model(image)
        loss = loss_fn(preds, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())
        targets_list.extend(targets.detach().cpu().numpy())

        pbar.set_description(f"Loss: {loss.item():.4f}")

    train_loss /= len(loader)
    train_acc = accuracy_score(targets_list, preds_list)
    train_f1 = f1_score(targets_list, preds_list, average='macro')

    ret = {
        "train_loss": train_loss,
        "train_acc": train_acc,
        "train_f1": train_f1,
    }

    # GPU 메모리 정리
    torch.cuda.empty_cache()
    
    return ret

## 3. Hyper-parameters
* 학습 및 추론에 필요한 하이퍼파라미터들을 정의합니다.

In [37]:
wandb.init(project = 'helloyoonjae-', name = 'fastcampus_cv11')

In [38]:
# device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# data config
#data_path = '/data/ephemeral/home/datasets_fin/'

# model config
model_name = 'efficientnet_b3.ra2_in1k' #'vit_base_patch16_224' #'vit_base_patch16_224.augreg_in1k' ##'densenet121.ra_in1k'# #'resnet101' #'resnet34' # 'resnet50' 'efficientnet-b0', ...

# training config
img_size = 256 #224 #256
LR = 1e-3
EPOCHS = 2
BATCH_SIZE = 32
num_workers = 0

In [39]:
# Hyper-parameters를 WandB에 로깅
wandb.config = {
    "learning_rate": LR,
    "epochs": EPOCHS,
    "batch_size": BATCH_SIZE,
    "model_name": model_name,
    "img_size": img_size,
    "num_workers": num_workers
}

## 4. Load Data
* 학습, 테스트 데이터셋과 로더를 정의합니다.

In [40]:
# augmentation을 위한 transform 코드
trn_transform = A.Compose([
    # 이미지 크기 조정
    A.Resize(height=img_size, width=img_size),
    # images normalization
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    # numpy 이미지나 PIL 이미지를 PyTorch 텐서로 변환
    ToTensorV2(),
])

# test image 변환을 위한 transform 코드
tst_transform = A.Compose([
    A.Resize(height=img_size, width=img_size),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2(),
])

# valid split

In [41]:
# Read the full dataset
full_dataset_df = pd.read_csv("/data/ephemeral/home/datasets_fin/combined_train.csv")

# Split the dataset into training and validation
train_df, valid_df = train_test_split(full_dataset_df, test_size=0.2, random_state=SEED)

# Save the split data (optional, for consistency in multiple runs)
train_df.to_csv("/data/ephemeral/home/datasets_fin/train_split.csv", index=False)
valid_df.to_csv("/data/ephemeral/home/datasets_fin/valid_split.csv", index=False)

In [42]:
# Dataset 정의
trn_dataset = ImageDataset(
    "/data/ephemeral/home/datasets_fin/combined_train.csv",
    "/data/ephemeral/home/datasets_fin/combined_train",
    transform=trn_transform
)
train_split_dataset = ImageDataset(
    "/data/ephemeral/home/datasets_fin/train_split.csv",
    "/data/ephemeral/home/datasets_fin/combined_train",
    transform=trn_transform  # Or a specific validation transform if different
)
val_split_dataset = ImageDataset(
    "/data/ephemeral/home/datasets_fin/valid_split.csv",
    "/data/ephemeral/home/datasets_fin/combined_train",
    transform=trn_transform  # Or a specific validation transform if different
)
tst_dataset = ImageDataset(
    "/data/ephemeral/home/datasets_fin/sample_submission.csv",
    "/data/ephemeral/home/datasets_fin/test/",
    transform=tst_transform
)
print(len(trn_dataset),len(train_split_dataset),len(val_split_dataset), len(tst_dataset))

15700 12560 3140 3140


In [43]:
# GPU 메모리 정리
torch.cuda.empty_cache()


# kfold

In [141]:
kf = KFold(n_splits=5, shuffle=True, random_state=SEED)

# K-Fold 교차 검증
fold = 0
for train_idx, val_idx in kf.split(trn_dataset):
    fold += 1
    print(f"Fold {fold}")

    train_subset = Subset(trn_dataset, train_idx)
    val_subset = Subset(trn_dataset, val_idx)

    train_loader = DataLoader(
        train_subset,
        batch_size=BATCH_SIZE,
        shuffle=True,
        num_workers=num_workers,
        pin_memory=True,
        drop_last=False
    )
    val_loader = DataLoader(
        val_subset,
        batch_size=BATCH_SIZE,
        shuffle=False,
        num_workers=0,
        pin_memory=True
    )

    # 모델 로드
    model = timm.create_model(
        model_name,
        pretrained=True,
        num_classes=17
    ).to(device)
    loss_fn = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr=LR)

    # 모델 학습
    for epoch in range(EPOCHS):
        ret = train_one_epoch(train_loader, model, optimizer, loss_fn, device)
        ret['epoch'] = epoch

        log = ""
        for k, v in ret.items():
            log += f"{k}: {v:.4f}\n"
        print(log)

        # 검증 세트에서 모델 평가
        model.eval()
        val_loss = 0
        preds_list = []
        targets_list = []

        with torch.no_grad():
            for image, targets in val_loader:
                image = image.to(device)
                targets = targets.to(device)

                preds = model(image)
                loss = loss_fn(preds, targets)

                val_loss += loss.item()
                preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())
                targets_list.extend(targets.detach().cpu().numpy())

        val_loss /= len(val_loader)
        val_acc = accuracy_score(targets_list, preds_list)
        val_f1 = f1_score(targets_list, preds_list, average='macro')
        print(f"Fold {fold}, Epoch {epoch+1}/{EPOCHS}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}, Val F1: {val_f1:.4f}")


Fold 1


Loss: 0.0064: 100%|██████████| 393/393 [01:59<00:00,  3.30it/s]


train_loss: 0.3318
train_acc: 0.8865
train_f1: 0.8786
epoch: 0.0000

Fold 1, Epoch 1/30, Val Loss: 0.0914, Val Acc: 0.9662, Val F1: 0.9649


Loss: 0.4254: 100%|██████████| 393/393 [01:57<00:00,  3.33it/s]


train_loss: 0.1117
train_acc: 0.9611
train_f1: 0.9592
epoch: 1.0000

Fold 1, Epoch 2/30, Val Loss: 0.1948, Val Acc: 0.9545, Val F1: 0.9524


...

train_loss: 0.0114
train_acc: 0.9973
train_f1: 0.9973
epoch: 27.0000

Fold 5, Epoch 28/30, Val Loss: 0.0287, Val Acc: 0.9914, Val F1: 0.9906


Loss: 0.0061: 100%|██████████| 393/393 [01:57<00:00,  3.33it/s]


train_loss: 0.0092
train_acc: 0.9970
train_f1: 0.9967
epoch: 28.0000

Fold 5, Epoch 29/30, Val Loss: 0.0248, Val Acc: 0.9930, Val F1: 0.9917


Loss: 0.0501: 100%|██████████| 393/393 [01:57<00:00,  3.33it/s]


train_loss: 0.0119
train_acc: 0.9967
train_f1: 0.9963
epoch: 29.0000

Fold 5, Epoch 30/30, Val Loss: 0.0709, Val Acc: 0.9783, Val F1: 0.9770


In [142]:
# 최종 전체 모델 학습
def train_final_model(train_loader, model, optimizer, loss_fn, device, epochs=30):
    scaler = GradScaler()
    for epoch in range(epochs):
        model.train()
        train_loss = 0
        preds_list = []
        targets_list = []

        pbar = tqdm(train_loader)
        for image, targets in pbar:
            image = image.to(device)
            targets = targets.to(device)

            optimizer.zero_grad(set_to_none=True)

            with autocast():
                preds = model(image)
                loss = loss_fn(preds, targets)

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            train_loss += loss.item()
            preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())
            targets_list.extend(targets.detach().cpu().numpy())

            pbar.set_description(f"Loss: {loss.item():.4f}")

        train_loss /= len(train_loader)
        train_acc = accuracy_score(targets_list, preds_list)
        train_f1 = f1_score(targets_list, preds_list, average='macro')

        ret = {
            "train_loss": train_loss,
            "train_acc": train_acc,
            "train_f1": train_f1,
        }

        log = ""
        for k, v in ret.items():
            log += f"{k}: {v:.4f}\n"
        print(log)
        
# 전체 데이터로 최종 모델 학습
final_dataset = ImageDataset(
    csv="/data/ephemeral/home/datasets_fin/combined_train.csv",
    path="/data/ephemeral/home/datasets_fin/combined_train",
    transform=trn_transform
)
final_loader = DataLoader(
    final_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=num_workers,
    pin_memory=True,
    drop_last=False
)

# 모델 로드
model = timm.create_model(
    model_name,
    pretrained=True,
    num_classes=17
).to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=LR)

# 최종 모델 학습
train_final_model(final_loader, model, optimizer, loss_fn, device, epochs=30)


# Kfold 끝

## 5. Train Model
* 모델을 로드하고, 학습을 진행합니다.

In [44]:
# valid

train_split_loader = DataLoader(
    train_split_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=num_workers,
    pin_memory=True,
    drop_last=False
)

val_split_loader = DataLoader(
    val_split_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,  # No need to shuffle validation data
    num_workers=0,
    pin_memory=True
)

# load model
model = timm.create_model(
    model_name,
    pretrained=True,
    num_classes=17
).to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=LR)


for epoch in range(EPOCHS):
    ret = train_one_epoch(train_split_loader, model, optimizer, loss_fn, device=device)
    ret['epoch'] = epoch

    log = ""
    for k, v in ret.items():
      log += f"{k}: {v:.4f}\n"
    print(log)
    
    # WandB에 로그 기록
    wandb.log({
        "epoch": epoch,
        "train_loss": ret["train_loss"],
        "train_acc": ret["train_acc"],
        "train_f1": ret["train_f1"]
    })
    
# 실험 종료
wandb.finish()


preds_list = []
targets_list = []

def evaluate_model(loader, model, device):
    model.eval()

    with torch.no_grad():
        for image, targets in tqdm(loader):
            image = image.to(device)
            targets = targets.to(device)
            preds = model(image)
            preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())
            targets_list.extend(targets.cpu().numpy())

    acc = accuracy_score(targets_list, preds_list)
    f1 = f1_score(targets_list, preds_list, average='macro')
    return {"accuracy": acc, "f1_score": f1}

preds_list

# Example of evaluating the model on the validation set
val_metrics = evaluate_model(val_split_loader, model, device)
print(f"Validation Accuracy: {val_metrics['accuracy']:.4f}, F1 Score: {val_metrics['f1_score']:.4f}")



Loss: 0.1047: 100%|██████████| 393/393 [01:25<00:00,  4.60it/s]


train_loss: 0.3504
train_acc: 0.8774
train_f1: 0.8686
epoch: 0.0000



Loss: 0.0158: 100%|██████████| 393/393 [01:25<00:00,  4.58it/s]


train_loss: 0.1027
train_acc: 0.9647
train_f1: 0.9625
epoch: 1.0000



epoch,▁█
train_acc,▁█
train_f1,▁█
train_loss,█▁
epoch,1
train_acc,0.96473
train_f1,0.96254
train_loss,0.1027


100%|██████████| 99/99 [00:14<00:00,  6.99it/s]

Validation Accuracy: 0.9662, F1 Score: 0.9656


In [45]:
pred_df = pd.DataFrame(val_split_dataset.df, columns=['ID', 'target'])
pred_df['valid_target'] = preds_list

sample_submission_df = pd.read_csv("/data/ephemeral/home/datasets_fin/valid_split.csv")
assert (sample_submission_df['ID'] == pred_df['ID']).all()

pred_df.to_csv("pred_0801_19:20.csv", index=False)

pred_df.head()

,ID,target,valid_target
0,cb21eaf0f58e325a_4.jpg,9,9
1,a7b4e66d98cde556_1.jpg,6,6
2,9223d8c34cef5b99_2_9.jpg,15,15
3,51ef555026d28c0d_1_7.jpg,12,12
4,258ffcf2fbc07d2c_1.jpg,3,4


In [61]:
pred_df = pd.read_csv(r'/data/ephemeral/home/notebook/pred_0801_19:20.csv')
meta = pd.read_csv(r'/data/ephemeral/home/datasets_fin/meta.csv')
pred_df = pred_df[pred_df['target'] != pred_df['valid_target']]
pred_df = pd.merge(pred_df, meta, on='target', how='inner')
pred_df.drop(columns='class_name', inplace=True)
pred_df = pred_df.rename(columns={
    'class_name_kr': 'target_class'
})
pred_df = pd.merge(pred_df, meta, left_on='valid_target', right_on='target', how='inner')
pred_df.drop(columns=['class_name','target_y'], inplace=True)
pred_df = pred_df.rename(columns={
    'class_name_kr': 'valid_target_class',
    'target_x': 'target'
})


In [66]:
pred_df[['target_class','valid_target_class']].value_counts()

target_class  valid_target_class
입퇴원확인서        통원진료확인서               26
통원진료확인서       진단서                   10
입퇴원확인서        진단서                    9
통원진료확인서       입퇴원확인서                 9
소견서           진단서                    9
이력서           자동차계기판                 5
입퇴원확인서        소견서                    5
외래진료비계산서영수증   진료비납입확인서               5
약제비영수증        진료비납입확인서               4
통원진료확인서       소견서                    3
입퇴원확인서        자동차등록증                 3
외래진료비계산서영수증   처방전                    2
입퇴원확인서        외래진료비계산서영수증            2
소견서           통원진료확인서                2
진료비납입확인서      진단서                    2
진단서           통원진료확인서                1
처방전           진료비납입확인서               1
통원진료확인서       자동차등록증                 1
계좌번호          자동차계기판                 1
주민등록증         약제비영수증                 1
입퇴원확인서        자동차계기판                 1
              약제비영수증                 1
약제비영수증        이력서                    1
              소견서                    1
입퇴원확인서        이력서              

In [37]:
# baseline
# DataLoader 정의
trn_loader = DataLoader(
    trn_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=num_workers,
    pin_memory=True,
    drop_last=False
)
tst_loader = DataLoader(
    tst_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=0,
    pin_memory=True
)

# load model
model = timm.create_model(
    model_name,
    pretrained=True,
    num_classes=17
).to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=LR)

for epoch in range(EPOCHS):
    ret = train_one_epoch(trn_loader, model, optimizer, loss_fn, device=device)
    ret['epoch'] = epoch

    log = ""
    for k, v in ret.items():
      log += f"{k}: {v:.4f}\n"
    print(log)
    
    # WandB에 로그 기록
    wandb.log({
        "epoch": epoch,
        "train_loss": ret["train_loss"],
        "train_acc": ret["train_acc"],
        "train_f1": ret["train_f1"]
    })
    
# 실험 종료
wandb.finish()

Loss: 0.2135: 100%|██████████| 491/491 [02:27<00:00,  3.33it/s]


train_loss: 0.2919
train_acc: 0.9024
train_f1: 0.8969
epoch: 0.0000



Loss: 0.0145: 100%|██████████| 491/491 [02:27<00:00,  3.34it/s]


train_loss: 0.0995
train_acc: 0.9668
train_f1: 0.9653
epoch: 1.0000



epoch,▁█
train_acc,▁█
train_f1,▁█
train_loss,█▁
epoch,1
train_acc,0.96675
train_f1,0.96526
train_loss,0.09953


In [84]:
import timm
print(timm.list_models(pretrained=True))


['bat_resnext26ts.ch_in1k', 'beit_base_patch16_224.in22k_ft_in22k', 'beit_base_patch16_224.in22k_ft_in22k_in1k', 'beit_base_patch16_384.in22k_ft_in22k_in1k', 'beit_large_patch16_224.in22k_ft_in22k', 'beit_large_patch16_224.in22k_ft_in22k_in1k', 'beit_large_patch16_384.in22k_ft_in22k_in1k', 'beit_large_patch16_512.in22k_ft_in22k_in1k', 'beitv2_base_patch16_224.in1k_ft_in1k', 'beitv2_base_patch16_224.in1k_ft_in22k', 'beitv2_base_patch16_224.in1k_ft_in22k_in1k', 'beitv2_large_patch16_224.in1k_ft_in1k', 'beitv2_large_patch16_224.in1k_ft_in22k', 'beitv2_large_patch16_224.in1k_ft_in22k_in1k', 'botnet26t_256.c1_in1k', 'caformer_b36.sail_in1k', 'caformer_b36.sail_in1k_384', 'caformer_b36.sail_in22k', 'caformer_b36.sail_in22k_ft_in1k', 'caformer_b36.sail_in22k_ft_in1k_384', 'caformer_m36.sail_in1k', 'caformer_m36.sail_in1k_384', 'caformer_m36.sail_in22k', 'caformer_m36.sail_in22k_ft_in1k', 'caformer_m36.sail_in22k_ft_in1k_384', 'caformer_s18.sail_in1k', 'caformer_s18.sail_in1k_384', 'caformer_s

# 6. Inference & Save File
* 테스트 이미지에 대한 추론을 진행하고, 결과 파일을 저장합니다.

In [38]:
preds_list = []

model.eval()
for image, _ in tqdm(tst_loader):
    image = image.to(device)

    with torch.no_grad():
        preds = model(image)
    preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())

100%|██████████| 99/99 [00:28<00:00,  3.45it/s]


In [45]:
preds.shape

torch.Size([4, 17])

In [46]:
pred_df = pd.DataFrame(tst_dataset.df, columns=['ID', 'target'])
pred_df['target'] = preds_list

In [47]:
sample_submission_df = pd.read_csv("/data/ephemeral/home/datasets_fin/sample_submission.csv")
assert (sample_submission_df['ID'] == pred_df['ID']).all()

In [48]:
pred_df.to_csv("pred_0801_09:15.csv", index=False)

In [49]:
pred_df.head()

,ID,target
0,0008fdb22ddce0ce.jpg,2
1,00091bffdffd83de.jpg,10
2,00396fbc1f6cc21d.jpg,5
3,00471f8038d9c4b6.jpg,12
4,00901f504008d884.jpg,2
